# Debugging autoreload

In [1]:
%load_ext autoreload
%autoreload 2

# Load packages

In [4]:
from pytorch_tabular.utils import load_covertype_dataset
from rich.pretty import pprint
from sklearn.model_selection import BaseCrossValidator, ParameterGrid, ParameterSampler
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import torch
import pickle
import shutil
from scipy import stats
import shap
from sklearn.model_selection import RepeatedStratifiedKFold
from glob import glob
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import itertools
from sklearn.model_selection import train_test_split
import numpy as np
from pytorch_tabular.utils import make_mixed_dataset, print_metrics
from pytorch_tabular import available_models
from pytorch_tabular import TabularModel
from pytorch_tabular.models import CategoryEmbeddingModelConfig, GANDALFConfig, TabNetModelConfig, FTTransformerConfig, DANetConfig
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig
from pytorch_tabular.models.common.heads import LinearHeadConfig
from pytorch_tabular.tabular_model_tuner import TabularModelTuner
from torchmetrics.functional.regression import mean_absolute_error, pearson_corrcoef
from pytorch_tabular import MODEL_SWEEP_PRESETS
import pandas as pd
from pytorch_tabular import model_sweep
from src.pt.model_sweep import model_sweep_custom
import warnings
from src.utils.configs import read_parse_config
from src.pt.hyper_opt import train_hyper_opt
from src.utils.hash import dict_hash
import pathlib
from tqdm import tqdm
import distinctipy
import matplotlib.patheffects as pe
import matplotlib.colors as mcolors
from statannotations.Annotator import Annotator
from scipy.stats import mannwhitneyu
from plottable import ColumnDefinition, Table
from plottable.plots import bar
from plottable.cmap import normed_cmap, centered_cmap
import optuna
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.cm
import matplotlib as mpl
from statsmodels.stats.multitest import multipletests
import re

# Load data

In [ ]:
path = f"D:/YandexDisk/Work/bbd/phenoage"
data = pd.read_excel(f"{path}/unn.xlsx", index_col=0)
data['NonLog C-reactive protein'] = data['C-reactive protein'].values
data['C-reactive protein'] = np.log(data['C-reactive protein'].values * 10.0) / np.log(10.0)
data.replace([np.inf, -np.inf], np.nan, inplace=True)
data.dropna(axis=0, how='any', inplace=True)
feats = pd.read_excel(f"{path}/phenoage.xlsx", index_col=0)
data['LinearComb'] = -19.9067
gamma = 0.0077
for f in feats.index.values:
    data['LinearComb'] += feats.at[f, 'Coeff'] * data[f].values
data['MortalityScore'] = 1 - np.exp(-np.exp(data['LinearComb'].values) * (np.exp(120 * gamma) - 1) / gamma)
data['PhenoAge'] = 141.50225 + np.log(-0.00553 * np.log(1 - data['MortalityScore'].values)) / 0.090165
data['PhenoAge acceleration'] = data['PhenoAge'] - data['Age']
data.to_excel(f"{path}/unn_result.xlsx", index_label="Features")

df_corr = pd.DataFrame(index=feats.index, columns=['rho', 'pval'])
for f in tqdm(feats.index.values):
    vals_1 = data.loc[:, 'PhenoAge'].values
    vals_2 = data.loc[:, f].values
    df_corr.at[f, 'rho'], df_corr.at[f, 'pval'] = stats.pearsonr(vals_1, vals_2)
_, df_corr['pval_fdr_bh'], _, _ = multipletests(df_corr.loc[:, 'pval'].values, 0.05, method='fdr_bh')
df_corr.sort_values(["rho"], ascending=[False], inplace=True)
df_corr.to_excel(f"{path}/age_pearson.xlsx", index_label="Features")

In [ ]:
fig = make_subplots(rows=1, cols=2, shared_yaxes=False, shared_xaxes=False, column_widths=[5, 3], horizontal_spacing=0.15)
min_plot_age = data[["Age", "PhenoAge"]].min().min()
max_plot_age = data[["Age", "PhenoAge"]].max().max()
shift_plot_age = max_plot_age - min_plot_age
min_plot_age -= 0.1 * shift_plot_age
max_plot_age += 0.1 * shift_plot_age
fig.add_trace(
    go.Scatter(
        x=[min_plot_age, max_plot_age],
        y=[min_plot_age, max_plot_age],
        showlegend=False,
        mode='lines',
        line = dict(color='black', width=2, dash='dot')
    ),
    row=1,
    col=1
)
fig.add_trace(
    go.Scatter(
        name='Scatter',
        x=data.loc[:, 'Age'].values,
        y=data.loc[:, 'PhenoAge'].values,
        text=data.index.values,
        hovertext=data.index.values,
        showlegend=False,
        mode='markers',
        marker=dict(
            size=10,
            opacity=0.75,
            line=dict(
                width=1,
                color='black'
            ),
            color='crimson'
        )
    ),
    row=1,
    col=1
)
fig.update_xaxes(
    row=1,
    col=1,
    automargin=True,
    title_text="Age",
    autorange=False,
    range=[min_plot_age, max_plot_age],
    showgrid=False,
    zeroline=False,
    linecolor='black',
    showline=True,
    gridcolor='gainsboro',
    gridwidth=0.05,
    mirror=True,
    ticks='outside',
    titlefont=dict(
        color='black',
        size=20
    ),
    showticklabels=True,
    tickangle=0,
    tickfont=dict(
        color='black',
        size=16
    ),
    exponentformat='e',
    showexponent='all'
)
fig.update_yaxes(
    row=1,
    col=1,
    automargin=True,
    title_text=f"PhenoAge",
    # scaleanchor="x",
    # scaleratio=1,
    autorange=False,
    range=[min_plot_age, max_plot_age],
    showgrid=False,
    zeroline=False,
    linecolor='black',
    showline=True,
    gridcolor='gainsboro',
    gridwidth=0.05,
    mirror=True,
    ticks='outside',
    titlefont=dict(
        color='black',
        size=20
    ),
    showticklabels=True,
    tickangle=0,
    tickfont=dict(
        color='black',
        size=16
    ),
    exponentformat='e',
    showexponent='all'
)
fig.add_trace(
    go.Violin(
        y=data.loc[:, 'PhenoAge acceleration'].values,
        hovertext=data.index.values,
        name="Violin",
        box_visible=True,
        meanline_visible=True,
        showlegend=False,
        line_color='black',
        fillcolor='crimson',
        marker=dict(color='crimson', line=dict(color='black', width=0.5), opacity=0.75),
        points='all',
        bandwidth=np.ptp(data.loc[:, 'PhenoAge acceleration'].values) / 32,
        opacity=0.75
    ),
    row=1,
    col=2
)
fig.update_yaxes(
    row=1,
    col=2,
    automargin=True,
    title_text="PhenoAge acceleraton",
    autorange=True,
    showgrid=False,
    zeroline=True,
    linecolor='black',
    showline=True,
    gridcolor='gainsboro',
    gridwidth=0.05,
    mirror=True,
    ticks='outside',
    titlefont=dict(
        color='black',
        size=20
    ),
    showticklabels=True,
    tickangle=0,
    tickfont=dict(
        color='black',
        size=16
    ),
    exponentformat='e',
    showexponent='all'
)
fig.update_xaxes(
    row=1,
    col=2,
    automargin=True,
    autorange=False,
    range=[-0.5, 0.3],
    showgrid=False,
    showline=True,
    zeroline=False,
    showticklabels=False,
    mirror=True,
    ticks='outside',
    tickvals=[],

)
fig.update_layout(
    template="simple_white",
    width=800,
    height=450,
    margin=go.layout.Margin(l=100, r=20, b=50, t=50, pad=0),
)
fig.show()
fig.write_image(f"{path}/unn.png")
fig.write_image(f"{path}/unn.pdf", format="pdf")